# Top Scorer - Network Analysis via Networks and Graph theory

<b>Summary:</b> We used networks and graph theory to understand the relationship between each ticket submitted. With this, we managed to map the unlinked customer 100% either via the phone number, email address or order id.

<b>Problem Statement:</b>
Customers can contact customer service via various channels such as the livechat function, filling up certain forms or calling in for help. Each time a customer contacts us with a new contact method, a new ticket is automatically generated. A complication arises when the same customer contacts us using different phone numbers or email addresses resulting in multiple tickets for the same issue. Hence, our challenge here is to identify how to merge relevant tickets together to create a complete picture of the customer issue and ultimately determine the RCR

<b>Dataset: </b>
You can download the dataset from https://www.kaggle.com/c/scl-2021-da/data

<b>Solution: </b>
The solution is simple and sweet, with minor cleaning and used basic network to map out the relationship between each ticket and we managed to formulate the solution and map out the customer network 100% within the timelimit.

### First we load the dataset

In [1]:
import pandas as pd
import networkx as nx

In [2]:
df = pd.read_json('dataset/contacts.json')
df

,Id,Email,Phone,Contacts,OrderId
0,0,gkzAbIy@qq.com,,1,
1,1,,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov
2,2,,9125983679,0,
3,3,mdllpYmE@gmail.com,,0,bHquEnCbbsGLqllwryxPsNOxa
4,4,,300364407,2,
...,...,...,...,...,...
499995,499995,,10072395382,2,whJlUOGNqjxCRzmIRdURQvlNv
499996,499996,,571709661031,4,JqIXOONvrwvJxZqNxCYHqnrKS
499997,499997,,4541459979,2,beXCZSzcHaBwAYoDcpQqjuAFO
499998,499998,RzSDsyH@hotmail.com,98947185431,1,ehjeFACGiwrERQxbziMxwOWku


## Data Cleaning
Data cleaning is the mandatory step before any modeling. 
We append "Email_ " , "Phone_" and "OrderId_"  to help us identify the data type later on.

In [3]:
df.Email = df.Email.apply(lambda x: "Email_"+x if x !='' else '')
df.Phone = df.Phone.apply(lambda x: "Phone_"+x if x !='' else '')
df.OrderId = df.OrderId.apply(lambda x: "OrderId_"+x if x !='' else '')
df

,Id,Email,Phone,Contacts,OrderId
0,0,Email_gkzAbIy@qq.com,,1,
1,1,,Phone_329442681752,4,OrderId_vDDJJcxfLtSfkooPhbYnJdxov
2,2,,Phone_9125983679,0,
3,3,Email_mdllpYmE@gmail.com,,0,OrderId_bHquEnCbbsGLqllwryxPsNOxa
4,4,,Phone_300364407,2,
...,...,...,...,...,...
499995,499995,,Phone_10072395382,2,OrderId_whJlUOGNqjxCRzmIRdURQvlNv
499996,499996,,Phone_571709661031,4,OrderId_JqIXOONvrwvJxZqNxCYHqnrKS
499997,499997,,Phone_4541459979,2,OrderId_beXCZSzcHaBwAYoDcpQqjuAFO
499998,499998,Email_RzSDsyH@hotmail.com,Phone_98947185431,1,OrderId_ehjeFACGiwrERQxbziMxwOWku


## Creating the Graph network
<b>1. Create the nodes with Id and with attributes as Contacts </b>

In [4]:
# Create the nodes with Id & Contacts

nodes = []

for _,Id,_,_,Contacts,_ in df.itertuples():
    nodes.append((Id,{"Contacts": Contacts}))

In [5]:
# Add the nodes into the new graph G

G = nx.Graph()
G.add_nodes_from(nodes)

In [6]:
# Now that we have 500000 nodes, 0 edges
G.number_of_nodes(), G.number_of_edges()

(500000, 0)

<b>2. Create the edges between: Id ↔ Email, Id ↔ Phone, Id ↔ OrderId</b>

And the nodes of Email, Phone & OrderId are auto created in the process.

In [7]:
# Create the edges between: Id ↔ Email, Id ↔ Phone, Id ↔ OrderId

G.add_edges_from(df[df.Email != ''][['Id', 'Email']].to_records(index=False))
G.add_edges_from(df[df.Phone != ''][['Id', 'Phone']].to_records(index=False))
G.add_edges_from(df[df.OrderId != ''][['Id', 'OrderId']].to_records(index=False))

In [8]:
G.number_of_nodes(), G.number_of_edges()

(1129135, 837231)

<b>3. List down the connected component.</b> 

Each list contains 1 connected component

In [9]:
# List down the connected component

conn_comp = list(nx.connected_components(G))
conn_comp

[{0, 'Email_gkzAbIy@qq.com'},
 {1,
  115061,
  140081,
  165605,
  2458,
  476346,
  98519,
  'Email_JmMSyjzmxdelSmeAHBUi@yahoo.com',
  'Email_ULziZaVD@hotmail.com',
  'Email_WXJDcOYGapCzchhwH@gmail.com',
  'Email_xXwrpkygOe@yahoo.com',
  'OrderId_mwVhJZGKtahXEdLMwVLcOAxXG',
  'OrderId_vDDJJcxfLtSfkooPhbYnJdxov',
  'Phone_069988936',
  'Phone_329442681752'},
 {159312,
  2,
  322639,
  348955,
  'Email_EQnHOWbVizLZxRnm@gmail.com',
  'OrderId_FTUBZYkUAEPIYmYvxtxKYYlqt',
  'OrderId_zDhCFyaTJEqyxtEqiHxdTPmPt',
  'Phone_5994952372',
  'Phone_9125983679'},
 {3, 'Email_mdllpYmE@gmail.com', 'OrderId_bHquEnCbbsGLqllwryxPsNOxa'},
 {4, 'Phone_300364407'},
 {212533,
  215197,
  226720,
  383605,
  404324,
  458692,
  482810,
  5,
  50,
  'Email_TIMejlVYhfqjy@gmail.com',
  'Email_izafiOnTIVibbz@hotmail.com',
  'Email_yFcCGIfYcFKfmqpNk@yahoo.com',
  'OrderId_XBkNrwcWWslDvSQdhYPfCpQJo',
  'OrderId_sEgewZTtnDMAADcwLSjxsHiSK',
  'Phone_04962058234',
  'Phone_2529522887',
  'Phone_840113148'},
 {142067,

## Extract the graph network 
The idea is one ticket_id one line, with their connected ticket id concated, number of contact points made summed up.

For each connected component, i.e. each list, we concated the ticket_id within, and summed up the contact made (stored under the nodes attributes earlier)

In [10]:
output = []

# for each connected component, i.e. each list, we apended the ticket_id within (to concat in the later part)

for each_connected_component in conn_comp:
    
    id_list = []

    for each_node in each_connected_component:
        
        # check if the node is a number, append to the id_list
        
        if str(each_node).isnumeric():

            id_list.append(each_node)

    sum_of_contacts = 0
    
    for order_id in id_list:
        
        # summed up the attributes ie. contact made that belongs to the node
        
        sum_of_contacts += G.nodes[order_id]['Contacts']

    output_str = '-'.join([str(each_node) for each_node in sorted(id_list)]) + ', '  + str(sum_of_contacts)
    for order_id in id_list:

        output.append([order_id, output_str])

Convert the output as dataframe, and sorted according to the competition requirement, and exported as csv file.

In [11]:
output_final = pd.DataFrame(output)
output_final= output_final.rename(columns={0:'ticket_id', 1:'ticket_trace/contact'})


In [12]:
output_final.sort_values('ticket_id').to_csv('output.csv', index=False)